In [7]:
pip install beautifulsoup4 lxml from bs4 import BeautifulSoup as BS
import pandas as pd
import requests
import time

# Define headers to simulate a real browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,/;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'https://www.google.com/',  # Adding a referer can help in bypassing restrictions
    'Connection': 'keep-alive'
}

# Function to get initial reviews and hotel data
def getReviews(url):
    response = requests.get(url, headers=headers)
    if response.status_code == 403:
        print("Access forbidden (403)")
        return None
    elif response.status_code != 200:
        print(f"Failed to fetch page, status code: {response.status_code}")
        return None
    html_code = response.text
    soup = BS(html_code, 'html.parser')

    # Extract the total number of reviews
    totalReviews = soup.find('span', class_='biGQs _P pZUbB KxBGd').text[:-8].replace(',', '')
    hotelName = soup.find('h1', id='HEADING').text

    # Extract reviews data
    reviewHeads = [i.text for i in soup.find_all('span', class_='JbGkU Cj')]
    reviewContent = [i.text for i in soup.find_all('span', class_='orRIx Ci _a C')]
    reviewerNdate = [i.text for i in soup.find_all('div', class_='tVWyV _Z o S4 H3 Ci') if 'wrote a review' in i.text]

    reviewerandDate = []
    for i in reviewerNdate:
        reviewerandDate.append(i.split('wrote a review '))
    finalDate = []
    finalreviewer = []
    for i in reviewerandDate:
        finalDate.append(i[1])
        finalreviewer.append(i[0])

    # Extract ratings and hotel location
    ratings = [i.find('title').text[0:3] for i in soup.find_all('div', class_='kmMXA _T Gi')]
 tempHotelLocation = [i.text for i in soup.find_all('span', class_='biGQs _P pZUbB KxBGd')]
    hotelLocation = tempHotelLocation[1]

    # Create DataFrame for the reviews
    df = pd.DataFrame({
        'Hotel Name': hotelName,
        'Hotel Location': hotelLocation,
        'Reviewer': finalreviewer,
        'Rating': ratings,
        'Date of Review': finalDate,
        'Review': reviewHeads,
        'Complete Review': reviewContent
    })

    return [df, totalReviews, hotelName, hotelLocation]

# Function to get additional reviews from paginated URLs
def getPureReviews(url, hotelName, hotelLocation):
    response = requests.get(url, headers=headers)
    if response.status_code == 403:
        print(f"Access forbidden (403) at {url}")
        return None
    elif response.status_code != 200:
        print(f"Failed to fetch page, status code: {response.status_code} at {url}")
        return None

    html_code = response.text
    soup = BS(html_code, 'html.parser')
# Extract reviews data
    reviewHeads = [i.text for i in soup.find_all('span', class_='JbGkU Cj')]
    reviewContent = [i.text for i in soup.find_all('span', class_='orRIx Ci _a C')]
    reviewerNdate = [i.text for i in soup.find_all('div', class_='tVWyV _Z o S4 H3 Ci') if 'wrote a review' in i.text]

    reviewerandDate = []
    for i in reviewerNdate:
        reviewerandDate.append(i.split('wrote a review '))
    finalDate = []
    finalreviewer = []
    for i in reviewerandDate:
        finalDate.append(i[1])
        finalreviewer.append(i[0])

    ratings = [i.find('title').text[0:3] for i in soup.find_all('div', class_='kmMXA _T Gi')]

    # Create DataFrame for the additional reviews
    df = pd.DataFrame({
        'Hotel Name': hotelName,
        'Hotel Location': hotelLocation,
        'Reviewer': finalreviewer,
        'Rating': ratings,
        'Date of Review': finalDate,
        'Review': reviewHeads,
        'Complete Review': reviewContent
    })

    return df # Main part of the script
url = "https://www.tripadvisor.com/Hotel_Review-g297683-d302143-Reviews-The_Oberoi_Amarvilas-Agra_Agra_District_Uttar_Pradesh.html"
split_url = url.split("Reviews-")
first_part = split_url[0] + "Reviews-or"
second_part = "-" + split_url[1]

# Get the first set of reviews and hotel data
result = getReviews(url)
if result is None:
    print("Failed to retrieve initial data.")
else:
    df, totalReviews, hotelName, hotelLocation = result

    # Generate URLs for paginated reviews
    urls = [f"{first_part}{i}{second_part}" for i in range(10, int(totalReviews)-10, 10)]

    # Fetch and concatenate reviews from additional pages
    for i in urls:
        newdf = getPureReviews(i, hotelName, hotelLocation)
        if newdf is not None:
            df = pd.concat([df, newdf], ignore_index=True)

    # Save the reviews to an Excel file
    df.to_excel('hotel_reviews1.xlsx', index=False, engine='openpyxl') files.dowmload('hotel_reviews1.xlsx')

IndentationError: unindent does not match any outer indentation level (<string>, line 47)